In [44]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [68]:
df_legitimate = pd.read_csv("legitimate_words.csv", header=None, encoding = 'utf8').dropna()
df_malicious = pd.read_csv("malicious_words.csv", header=None, encoding = 'utf8').dropna()
X_data = df_legitimate[0].tolist() + df_malicious[0].tolist()
y_data = np.zeros(len(df_legitimate)).tolist() + np.ones(len(df_malicious)).tolist()

In [91]:
df_legitimate = pd.read_csv("legitimate.txt", names=['url'], encoding = 'utf8').dropna()
df_malicious = pd.read_csv("malicious.txt",names=['url'], encoding = 'utf8').dropna()
X_data = df_legitimate['url'].tolist() + df_malicious['url'].tolist()
y_data = np.zeros(len(df_legitimate)).tolist() + np.ones(len(df_malicious)).tolist()

#dfdf==pdpd..read_csvread_csv('smsspam',sep='\t',names=['Status','Message'])

In [90]:
type(df_legitimate['url'].tolist())

list

In [105]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0, shuffle = True)

In [106]:
#cv_test = CountVectorizer()
#x_traincv_test = cv_test.fit_transform(["Hi How are you How are you doing","Hi what's up","Wow that's awesome ttt/213s>2ss"])
#x_traincv_test.toarray()
#cv_test.get_feature_names()


In [107]:
cv = CountVectorizer()
tf_idf = TfidfVectorizer()
x_traincv =cv.fit_transform(X_train)
x_testcv=cv.transform(X_test)
x_train_tf_idf =tf_idf.fit_transform(X_train)
x_test_tf_idf=tf_idf.transform(X_test)

y_train= [int(i) for i in y_train]
y_test= [int(i) for i in y_test]

In [108]:
x_train_tf_idf[0].toarray()[x_train_tf_idf[0].toarray()!=0]
np.array(tf_idf.get_feature_names())

array(['00', '000', '0000', ..., '㸀㰀script㸀confirm', 'ﬁnd', 'ｅｘｐｒｅｓｓｉｏｎ'],
      dtype='<U1026')

In [109]:
#top features
indices = np.argsort(tf_idf.idf_)[::-1]
features = tf_idf.get_feature_names()
top_n = 20
top_features = [features[i] for i in indices[:top_n]]
print (top_features)

['ｅｘｐｒｅｓｓｉｏｎ', 'alumni_off', 'aluvinmisery_vclip', 'alunos', 'alundraii', 'alundainaward', 'alumpostcards2', 'alumphoto', 'alumnos', 'alumnisociety', 'alumnisignup', 'alumniresources', 'alumnireception2006', 'alumnioff', 'alumnifriends', 'alumnicenter', 'alumni_update', 'alumni_strategy', 'alumni_smile', 'alumni_pride']


In [110]:
len(cv.get_feature_names())

753457

In [111]:
mnb_model = MultinomialNB()
linear_svc_model = LinearSVC()
logistic_model = LogisticRegression()

In [119]:
mnb_model.fit(x_traincv, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [120]:
linear_svc_model.fit(x_traincv, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [121]:
logistic_model.fit(x_traincv, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [122]:
def confusion(model, X,y):
    predictions=model.predict(X)

    tn, fp, fn, tp = confusion_matrix(predictions, y).ravel()
    total = tn+fp+fn+tp
    actual_no = tn + fp
    actual_yes = tp + fn

    #print(tn, fp, fn, tp)
    #print((tn + tp) / (tn+fp+fn+tp))
    #print(tp / (tp + fn))
    #print((tp + fp) / (tn+fp+fn+tp))
    #print(tp / (tp+fp))
    #print(tn / (tn+fn))
    print('tn:' + str(tn), 'fp:' + str(fp))
    print('fn:' + str(fn), 'tp:' + str(tp) +'\n')
    print('Accuracy: ' + str((tp+tn)*100/total))
    print('Misclassification: ' + str((fp+fn)*100/total))
    print('True Positive Rate: ' + str((tp)*100/actual_yes))
    print('False Positive Rat: ' + str((fp)*100/actual_no))
    print('Specificity: ' + str((tn)*100/actual_no))
    print('Precision: ' + str((tp)*100/(tp+fp)))
    print('Prevalence: ' + str(actual_yes*100/total))
    print(tp / (tp+fp))
    print(tn / (tn+fn))

In [123]:
confusion(mnb_model, x_testcv, y_test)

tn:258471 fp:2079
fn:489 tp:7260

Accuracy: 99.04285889995863
Misclassification: 0.9571411000413718
True Positive Rate: 93.68950832365466
False Positive Rat: 0.7979274611398963
Specificity: 99.20207253886011
Precision: 77.73851590106007
Prevalence: 2.888195632484653
0.7773851590106007
0.9981116774791473


In [124]:
confusion(linear_svc_model, x_testcv, y_test)

tn:258594 fp:535
fn:366 tp:8804

Accuracy: 99.66418063429234
Misclassification: 0.33581936570766197
True Positive Rate: 96.00872410032716
False Positive Rat: 0.206460874699474
Specificity: 99.79353912530053
Precision: 94.27133526073456
Prevalence: 3.417828616580755
0.9427133526073456
0.9985866543095459


In [125]:
confusion(logistic_model, x_testcv, y_test)

tn:258603 fp:690
fn:357 tp:8649

Accuracy: 99.60976373374481
Misclassification: 0.3902362662551854
True Positive Rate: 96.03597601598933
False Positive Rat: 0.26610822505813886
Specificity: 99.73389177494187
Precision: 92.61162865403148
Prevalence: 3.356702783089016
0.9261162865403149
0.9986214087117702
